In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd

import os
# PACKAGE_FOLDER =  os.environ.get('PACKAGE_FOLDER')
PACKAGE_FOLDER = '/Users/amash/Insync/GDrive/Projects/Raptor/raptor_functions'
sys.path.insert(0, PACKAGE_FOLDER)

# Libraries to install

In [2]:
# !pip install mlflow boto3 awscli tsfresh raptor_functions

# Retrieve Model URI from mlflow

In [3]:
# !aws configure

This is the only thing we add in the code to make a prediction + new data

In [4]:
# INPUT - URI to the MLFlow experiment
# ************************************
# ************************************
model_uri = 's3://raptor-mlflow-data/mlartifacts/3/24ed79470750408cb81dde2c7a1845b4/artifacts/ssc'
# ************************************
# ************************************

## Get new data

In [5]:
# model_uri1 = 's3://raptor-mlflow-data/mlartifacts/1/5c313b84ccfa4c39a538a1d3241cafab/artifacts/Extreme Gradient Boosting'

In [4]:
model_uri1 = "s3://raptor-mlflow-data/mlartifacts/1/db53e8baeb3f4ecb9116f13b033ec75e/artifacts/Extreme Gradient Boosting"
model_uri2 = 's3://raptor-mlflow-data/mlartifacts/3/e0ae99ed8de249feab80393307c9be22/artifacts/ssc'
# model_uri2 = 's3://raptor-mlflow-data/mlartifacts/3/3032491ee7e44b59a99c3209716af01a/artifacts/ssc'


In [5]:
from raptor_functions.supervised.datasets import get_data
df = get_data('handheld_data')

# select one single experiment
df_exp = df.groupby('exp_unique_id').get_group(0)

# Predict on new data

### Method 1
- get prediction data
- load model
- make prediction

In [6]:
# from raptor_functions.supervised.prediction import get_prediction_features

from raptor_functions.semi_supervised.prediction import get_prediction_features

prediction_data1 = get_prediction_features(df_exp, model_uri1)
prediction_data2 = get_prediction_features(df_exp, model_uri2)

Feature Extraction: 100%|██████████| 18/18 [00:00<00:00, 3628.99it/s]
/Users/amash/Insync/GDrive/Projects/Raptor/raptor_functions/raptor_functions/semi_supervised/prediction.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  prediction_data.columns = prediction_data.columns.str.replace('["]', "")
Feature Extraction: 100%|██████████| 18/18 [00:00<00:00, 8538.51it/s]
/Users/amash/Insync/GDrive/Projects/Raptor/raptor_functions/raptor_functions/semi_supervised/prediction.py:60: FutureWarning: The default value of regex will change from True to False in a future version.
  prediction_data.columns = prediction_data.columns.str.replace('["]', "")


In [7]:
prediction_data1

,sensor_5__fft_coefficient__attr_angle__coeff_34
0,27.346596


In [8]:
prediction_data2

,sensor_1__autocorrelation__lag_9
0,0.679405


In [9]:
import mlflow
# Load model as a PyFuncModel.
loaded_model1 = mlflow.pyfunc.load_model(model_uri1)
pred1 = loaded_model1.predict(prediction_data1)

loaded_model2 = mlflow.pyfunc.load_model(model_uri2)
pred2 = loaded_model2.predict(prediction_data2)

In [10]:
pred1

array([0])

In [11]:
pred2

0

### Method 2
- make prediction

In [12]:
from raptor_functions.supervised.prediction import make_prediction, make_ensemble_prediction


In [13]:
df_exp

,exp_unique_id,exp_name,timesteps,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,...,sensor_23,sensor_24,Humidity (r.h.),measurement_stage,date_exp,time_elapsed,datetime_exp_start,datetime_exp,filename,result
0,0,Test 24,1,131.938,132.443,177.444,178.157,44.974,44.508,219.374,...,143.288,142.050,46.07,baseline,21/3/2022,0.00,14:12:54,2022-03-21 14:12:54.000,5 NegTest_3_20220321_14_12.txt,Control
1,0,Test 24,2,133.078,133.972,177.444,178.694,45.131,44.611,219.607,...,142.874,141.914,46.07,baseline,21/3/2022,0.25,14:12:54,2022-03-21 14:12:54.250,5 NegTest_3_20220321_14_12.txt,Control
2,0,Test 24,3,131.812,132.064,177.622,178.694,45.183,44.663,219.841,...,143.288,141.914,46.07,baseline,21/3/2022,0.50,14:12:54,2022-03-21 14:12:54.500,5 NegTest_3_20220321_14_12.txt,Control
3,0,Test 24,4,131.059,132.064,177.444,178.694,45.183,44.663,219.607,...,142.599,141.368,46.07,baseline,21/3/2022,0.75,14:12:54,2022-03-21 14:12:54.750,5 NegTest_3_20220321_14_12.txt,Control
4,0,Test 24,5,131.435,131.938,177.622,178.694,45.131,44.663,219.607,...,142.187,140.825,46.07,baseline,21/3/2022,1.00,14:12:54,2022-03-21 14:12:55.000,5 NegTest_3_20220321_14_12.txt,Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0,Test 24,124,132.064,132.824,177.444,178.514,45.026,44.508,219.841,...,142.462,141.368,46.00,flush,21/3/2022,30.75,14:12:54,2022-03-21 14:13:24.750,5 NegTest_3_20220321_14_12.txt,Control
124,0,Test 24,125,131.812,132.697,177.444,178.514,45.131,44.611,219.607,...,140.690,139.478,46.07,flush,21/3/2022,31.00,14:12:54,2022-03-21 14:13:25.000,5 NegTest_3_20220321_14_12.txt,Control
125,0,Test 24,126,132.570,133.078,177.444,178.336,45.131,44.560,219.841,...,141.504,140.150,46.07,flush,21/3/2022,31.25,14:12:54,2022-03-21 14:13:25.250,5 NegTest_3_20220321_14_12.txt,Control
126,0,Test 24,127,132.443,132.824,177.267,178.336,45.079,44.508,219.607,...,140.961,139.881,46.07,flush,21/3/2022,31.50,14:12:54,2022-03-21 14:13:25.500,5 NegTest_3_20220321_14_12.txt,Control


In [14]:
df_exp.ndim

2

In [15]:

pred1 = make_prediction(df_exp, model_uri1)

Feature Extraction: 100%|██████████| 18/18 [00:00<00:00, 4809.06it/s]
/Users/amash/Insync/GDrive/Projects/Raptor/raptor_functions/raptor_functions/supervised/prediction.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  prediction_data.columns = prediction_data.columns.str.replace('["]', "")


In [16]:
pred1

array([0])

In [17]:
pred2 = make_prediction(df_exp, model_uri2)

Feature Extraction: 100%|██████████| 18/18 [00:00<00:00, 3666.53it/s]
/Users/amash/Insync/GDrive/Projects/Raptor/raptor_functions/raptor_functions/supervised/prediction.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  prediction_data.columns = prediction_data.columns.str.replace('["]', "")


In [18]:
pred2

0

In [19]:
d = make_ensemble_prediction(df_exp)

Feature Extraction: 100%|██████████| 18/18 [00:00<00:00, 998.10it/s]
/Users/amash/Insync/GDrive/Projects/Raptor/raptor_functions/raptor_functions/supervised/prediction.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  prediction_data.columns = prediction_data.columns.str.replace('["]', "")
Feature Extraction: 100%|██████████| 18/18 [00:00<00:00, 975.97it/s]
/Users/amash/Insync/GDrive/Projects/Raptor/raptor_functions/raptor_functions/supervised/prediction.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  prediction_data.columns = prediction_data.columns.str.replace('["]', "")
Feature Extraction: 100%|██████████| 18/18 [00:00<00:00, 2233.32it/s]

type preds:  <class 'dict'>
preds:  {'Extra Trees Classifier': array([0]), 'Logistic Regression': array([1]), 'ssc': 0}



/Users/amash/Insync/GDrive/Projects/Raptor/raptor_functions/raptor_functions/supervised/prediction.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  prediction_data.columns = prediction_data.columns.str.replace('["]', "")


In [20]:
d

(0,
 {'Extra Trees Classifier': array([0]),
  'Logistic Regression': array([1]),
  'ssc': 0})

In [22]:
from raptor_functions.supervised.preprocess import preprocess_single_file

In [26]:
filepath = '/content/drive/MyDrive/Projects/Raptor/data/Raw_Data/24H 1 Control_14_15_26.28.txt'

In [28]:
preprocess_single_file(filepath)

,exp_unique_id,timesteps,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,humidity,measurement_stage,date_exp,time_elapsed,datetime_exp_start,datetime_exp,filename,exp_name,repeat_no,result
0,0,0.0,314.851,310.005,305.016,303.225,259.226,303.750,305.708,313.180,298.355,299.501,303.631,308.597,104.813,109.898,114.768,116.487,160.629,115.508,113.813,105.887,120.402,119.256,114.887,110.686,30.810,baseline,13/07/2021,0.00,14:16:06,2021-07-13 14:16:06.000,24H 1 Control_14_15_26.28.txt,24H 1 Control,2,Control
1,0,1.0,314.899,309.623,304.323,302.151,258.391,303.655,305.851,313.180,298.785,299.740,303.703,308.453,104.980,109.993,114.935,116.463,160.677,115.484,113.932,105.911,120.498,119.232,114.744,110.590,42.992,baseline,13/07/2021,0.25,14:16:06,2021-07-13 14:16:06.250,24H 1 Control_14_15_26.28.txt,24H 1 Control,2,Control
2,0,2.0,314.493,309.647,304.896,302.533,258.391,304.084,305.708,313.085,298.426,299.525,303.726,308.716,104.980,109.898,114.816,116.415,160.701,115.389,113.765,105.887,120.450,119.089,114.887,110.829,51.970,baseline,13/07/2021,0.50,14:16:06,2021-07-13 14:16:06.500,24H 1 Control_14_15_26.28.txt,24H 1 Control,2,Control
3,0,3.0,314.708,310.196,304.848,302.604,258.295,303.607,305.517,313.371,299.071,299.907,303.989,308.788,105.075,109.826,114.673,116.487,160.629,115.484,114.052,105.935,120.450,119.137,114.816,110.614,47.278,baseline,13/07/2021,0.75,14:16:06,2021-07-13 14:16:06.750,24H 1 Control_14_15_26.28.txt,24H 1 Control,2,Control
4,0,4.0,314.708,309.910,304.562,302.103,258.701,303.726,305.827,313.324,299.238,299.811,304.180,308.525,105.099,109.874,114.816,116.320,160.533,115.389,113.765,105.887,120.283,119.185,114.768,110.662,35.141,baseline,13/07/2021,1.00,14:16:06,2021-07-13 14:16:07.000,24H 1 Control_14_15_26.28.txt,24H 1 Control,2,Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,0,116.0,314.302,309.766,304.538,302.437,258.128,302.724,304.705,312.894,297.209,298.737,303.321,308.859,105.147,110.137,115.174,116.559,161.536,116.272,114.696,106.174,122.073,120.092,115.317,110.948,50.843,flush,13/07/2021,29.00,14:16:06,2021-07-13 14:16:35.000,24H 1 Control_14_15_26.28.txt,24H 1 Control,2,Control
117,0,117.0,314.565,309.575,304.323,302.389,258.128,303.321,304.872,313.037,297.567,298.928,303.416,308.597,105.052,110.256,115.246,116.654,161.512,116.368,114.625,106.006,122.049,119.996,115.413,110.948,40.330,flush,13/07/2021,29.25,14:16:06,2021-07-13 14:16:35.250,24H 1 Control_14_15_26.28.txt,24H 1 Control,2,Control
118,0,118.0,314.947,309.790,304.299,302.127,257.889,302.795,304.968,312.965,297.352,299.167,303.416,308.597,105.266,110.065,115.269,116.630,161.512,116.272,114.696,106.078,121.978,120.163,115.436,110.972,29.276,flush,13/07/2021,29.50,14:16:06,2021-07-13 14:16:35.500,24H 1 Control_14_15_26.28.txt,24H 1 Control,2,Control
119,0,119.0,314.016,309.647,304.347,302.533,257.651,302.652,304.825,312.774,297.257,299.262,303.726,308.501,105.362,110.089,115.126,116.582,161.393,116.463,114.601,105.863,121.715,119.805,115.556,111.092,30.358,flush,13/07/2021,29.75,14:16:06,2021-07-13 14:16:35.750,24H 1 Control_14_15_26.28.txt,24H 1 Control,2,Control
